In [12]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [13]:
img_size = 96
epochs = 75
batch_size = 48
testsplit = .2
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)

23


In [14]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
    validation_split=testsplit,
        preprocessing_function=preprocess_input
)

female_val_generator = train_datagen.flow_from_directory(
        "CK+MF/Female",
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed,
     subset="validation",
        
)

male_val_generator = train_datagen.flow_from_directory(
        "CK+MF/Male",
        target_size=(targetx, targety),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
        seed=seed,
     subset="validation",
        
)

Found 121 images belonging to 7 classes.
Found 72 images belonging to 7 classes.


In [15]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import classification_report
import time

def evaluate_keras_model(keras_model, data_generator, report_prefix="classification_report"):
    """
    This function takes a Keras model and a data generator as inputs, predicts the classes,
    and prints a classification report. The report variable changes with each function run.

    Args:
    keras_model (tensorflow.keras.Model): A compiled Keras model.
    data_generator (tf.keras.utils.Sequence): A Keras data generator for validation/testing.
    report_prefix (str): Prefix for the classification report variable name.

    Returns:
    dict: A dictionary containing the classification report with a unique key.
    """
    # Make predictions using the provided Keras model
    predictions = keras_model.predict(data_generator, steps=len(data_generator))
    
    # Get the predicted class indices
    y_pred = np.argmax(predictions, axis=1)

    # Generate a unique identifier for the report name
    timestamp = int(time.time())
    unique_cr_name = f"{report_prefix}_{timestamp}"

    # Print and return the classification report
    print('Classification Report')
    classification_report_result = classification_report(
        y_true=data_generator.classes,
        y_pred=y_pred,
        target_names=list(data_generator.class_indices.keys())
    )
    print(classification_report_result)

    # Store the classification report in a dictionary with a unique key
    results = {unique_cr_name: classification_report_result}
    
    return results

# Example setup:
models = [
    tf.keras.models.load_model('DNetCK+Femalemodel.keras'),
    tf.keras.models.load_model('MNV2CKFemalemodel.keras'),
    tf.keras.models.load_model('RNCK+Female.keras'),
    tf.keras.models.load_model('SeqCK+Femalemodel.keras'),
    tf.keras.models.load_model('IV3CK+Femalemodel.keras')
    
]

# Assuming `data_generator` is already defined and is shared among models
data_generator = male_val_generator

# Loop through models and use the same data generator
for i, model in enumerate(models, start=1):
    report_prefix = f"report_model_{i}"
    results = evaluate_keras_model(model, data_generator, report_prefix=report_prefix)

2/2 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step
Classification Report
              precision    recall  f1-score   support

       anger       1.00      0.64      0.78        11
    contempt       0.50      1.00      0.67         3
     disgust       0.89      1.00      0.94        17
        fear       0.00      0.00      0.00         4
       happy       1.00      0.54      0.70        13
     sadness       0.38      0.50      0.43         6
    surprise       0.72      1.00      0.84        18

    accuracy                           0.76        72
   macro avg       0.64      0.67      0.62        72
weighted avg       0.78      0.76      0.74        72

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 999ms/step
Classification Report
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        11
    contempt       0.00      0.00      0.00         3
     disgust       0.24      1.00      0.38        17
        fear       0.00      0.00      0.00         4
       happy 

In [16]:
Y_pred = DNetKDEFMale.predict(male_val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true=male_val_generator.classes

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
cm

labels=['angry','disgust','fear','happy','neutral','sad','surprise']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()

NameError: name 'DNetKDEFMale' is not defined

In [ ]:
Y_pred = DNetKDEFFemale.predict(male_val_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true=male_val_generator.classes

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_true,y_pred)
cm

labels=['angry','disgust','fear','happy','neutral','sad','surprise']
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.show()